In [581]:
import pandas as pd
from tqdm.notebook import tqdm
from os import listdir
from os.path import isfile, join
import shutil
import plotly.express as px
import random
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import MinMaxScaler

## Extracting properties of airfoils

In [88]:
pathForCharacteristics = '/Users/inotin/Dropbox/DS/Personal/airfoil/TR824-Digitized/'

In [97]:
filesInDirectory = [f for f in listdir(pathForCharacteristics) if isfile(join(pathForCharacteristics, f)) and f.split('.')[1]=='ALL']

In [105]:
def extractProperties(filePath, parameter='Alpha, Cl, Cd(interpolated)', num = 1):
    f = open(filePath, "r")
    f = f.read()
    
    dct = {'id':[],'alpha':[],'Cl':[],'Cd':[]}
    for r in f.split(parameter)[1].split('Data set')[1:][num].split('\n')[2:-1]:
        row = [x for x in r.split(' ') if x!='']
        dct['id'].append(filePath.split('/')[-1].split('.')[0])
        dct['alpha'].append(float(row[0]))
        dct['Cl'].append(float(row[1]))
        dct['Cd'].append(float(row[2]))
    return pd.DataFrame(dct)

In [156]:
dfProperties = pd.DataFrame()
for file in filesInDirectory:
    dfProperties = pd.concat([dfProperties, extractProperties(pathForCharacteristics+file)])
dfProperties

,id,alpha,Cl,Cd
0,216,19.37620,1.373150,0.017005
1,216,18.29290,1.411050,0.018944
2,216,17.40130,1.519540,0.000000
3,216,16.28550,1.549260,0.000000
4,216,15.29650,1.531330,0.000000
...,...,...,...,...
24,233,-8.23054,-0.714753,0.010010
25,233,-10.40240,-0.904422,0.000000
26,233,-11.07290,-0.985190,0.000000
27,233,-12.35700,-0.646564,0.009324


## Forming dataframe with ids and airfoil names (type, model)

In [175]:
f = open(path+"NACA.LST", "r")
f=f.read()

In [176]:
dct = {'id':[],'type':[],'model':[]}
for r in f.split('---------------------')[1].split('\n')[1:-1]:
    dct['id'].append(r.split('  ')[0].split('.')[0][1:])
    dct['type'].append(r.split('  ')[1])
    dct['model'].append(r.split('  ')[2])
dfNames = pd.DataFrame(dct)
dfNames.head()

,id,type,model
0,131,NACA,0006
1,132,NACA,0009
2,133,NACA,1408
3,134,NACA,1410
4,135,NACA,1412


## Forming a data frame with airfoil geometries

In [178]:
import urllib.request
from requests import get  # to make GET request


def download(url, file_name):
    response = get(url)
    if response.status_code != 404:
        # open in binary mode
        with open(file_name, "wb") as file:
            # get request
            file.write(response.content)


### Downloading airfoli geometries to coords folder

In [179]:
ids = dfNames['id'].values
for i,r in tqdm(dfNames.iterrows(), total = len(dfNames)):
    mdl = ''.join(c for c in r['model'] if c.isdigit())
    download(f"https://m-selig.ae.illinois.edu/ads/coord/n{r['model']}.dat", 
             f"/Users/inotin/Dropbox/DS/Personal/airfoil/coords/{r['id']}.dat")
    download(f"https://m-selig.ae.illinois.edu/ads/coord/naca{r['model']}.dat", 
             f"/Users/inotin/Dropbox/DS/Personal/airfoil/coords/{r['id']}.dat")
    download(f"https://m-selig.ae.illinois.edu/ads/coord/naca{mdl}.dat", 
             f"/Users/inotin/Dropbox/DS/Personal/airfoil/coords/{r['id']}.dat")

In [417]:
mypath='/Users/inotin/Dropbox/DS/Personal/airfoil/coords/'
files = [f for f in listdir(mypath) if isfile(join(mypath, f)) and f.split('.')[1]=='dat']
print(f'Succesfully downloaded {len(files)} from {len(dfNames)}')

Succesfully downloaded 64 from 118


### Forming a data frame

In [497]:
import re
dfCoords = pd.DataFrame()
for file in files:
    f = open(mypath+file, "r")
    f = f.read()
    xs = []
    ys = []

    for r in f.split('\n')[1:-1]:
        row = re.findall("-?\d+\.\d+",r)
        if len(row)==2:
            xs.append(float(row[0]))
            ys.append(float(row[1]))
    dfCoords = pd.concat([dfCoords, pd.DataFrame({'id':[file.split('.')[0]],
                                                 'xs':[xs], 
                                                 'ys':[ys]})])

## Joining all together

In [552]:
df2 = dfProperties.merge(dfCoords, left_on='id', right_on='id', how='left')
df2.dropna(inplace=True)

In [553]:
df2.head()

,id,alpha,Cl,Cd,xs,ys
0,216,19.3762,1.37315,0.017005,"[1.0, 0.9502, 0.9004, 0.85055, 0.80063, 0.7506...","[0.0, 0.00744, 0.0166, 0.02649, 0.03653, 0.046..."
1,216,18.2929,1.41105,0.018944,"[1.0, 0.9502, 0.9004, 0.85055, 0.80063, 0.7506...","[0.0, 0.00744, 0.0166, 0.02649, 0.03653, 0.046..."
2,216,17.4013,1.51954,0.000000,"[1.0, 0.9502, 0.9004, 0.85055, 0.80063, 0.7506...","[0.0, 0.00744, 0.0166, 0.02649, 0.03653, 0.046..."
3,216,16.2855,1.54926,0.000000,"[1.0, 0.9502, 0.9004, 0.85055, 0.80063, 0.7506...","[0.0, 0.00744, 0.0166, 0.02649, 0.03653, 0.046..."
4,216,15.2965,1.53133,0.000000,"[1.0, 0.9502, 0.9004, 0.85055, 0.80063, 0.7506...","[0.0, 0.00744, 0.0166, 0.02649, 0.03653, 0.046..."


## Some visuals

In [580]:
idx='216'
sampleAlphaLiftDrag=dfProperties[dfProperties['id']==idx]

xs = dfCoords[dfCoords['id']==idx]['xs'].values[0]
ys = dfCoords[dfCoords['id']==idx]['ys'].values[0]
title=dfNames[dfNames['id']==idx].values[0][1] +' '+\
dfNames[dfNames['id']==idx].values[0][2]
f=px.line(x=xs, y=ys, title='<b>'+title+'</b>')
f.update_yaxes(scaleanchor = "x", scaleratio = 1)
f.show()

f = px.line(sampleAlphaLiftDrag, x ='alpha', y='Cl', title=f'Lift coefficient (Cl) vs Angle of attack (alpha) for <b>{title}</b>')
f.show()
f = px.line(sampleAlphaLiftDrag, x ='alpha', y='Cd', title=f'Lift coefficient (Cl) vs Angle of attack (alpha) for <b>{title}</b>')
f.show()

## Feature generation
As soon as x coordinates of shape do not have any valuable information, I decided to find a ratio of y coordinates to x coordinates.  
Later I will need to remove inf values as there are 0's in denominator. This new feature can be considered as a parameter defining angle of a vector connecting the origin with the point and therefore represents geometry of an airfoil. 

In [554]:
df2['xs'] = df2['xs'].apply(lambda x: np.array(x))
df2['ys'] = df2['ys'].apply(lambda x: np.array(x))
df2['ysr'] = df2['ys']/df2['xs']
df2

,id,alpha,Cl,Cd,xs,ys,ysr
0,216,19.37620,1.373150,0.017005,"[1.0, 0.9502, 0.9004, 0.85055, 0.80063, 0.7506...","[0.0, 0.00744, 0.0166, 0.02649, 0.03653, 0.046...","[0.0, 0.00782993054093875, 0.01843625055530875..."
1,216,18.29290,1.411050,0.018944,"[1.0, 0.9502, 0.9004, 0.85055, 0.80063, 0.7506...","[0.0, 0.00744, 0.0166, 0.02649, 0.03653, 0.046...","[0.0, 0.00782993054093875, 0.01843625055530875..."
2,216,17.40130,1.519540,0.000000,"[1.0, 0.9502, 0.9004, 0.85055, 0.80063, 0.7506...","[0.0, 0.00744, 0.0166, 0.02649, 0.03653, 0.046...","[0.0, 0.00782993054093875, 0.01843625055530875..."
3,216,16.28550,1.549260,0.000000,"[1.0, 0.9502, 0.9004, 0.85055, 0.80063, 0.7506...","[0.0, 0.00744, 0.0166, 0.02649, 0.03653, 0.046...","[0.0, 0.00782993054093875, 0.01843625055530875..."
4,216,15.29650,1.531330,0.000000,"[1.0, 0.9502, 0.9004, 0.85055, 0.80063, 0.7506...","[0.0, 0.00744, 0.0166, 0.02649, 0.03653, 0.046...","[0.0, 0.00782993054093875, 0.01843625055530875..."
...,...,...,...,...,...,...,...
3318,233,-8.23054,-0.714753,0.010010,"[1.0, 0.95022, 0.90043, 0.85057, 0.80065, 0.75...","[0.0, 0.00789, 0.0175, 0.02755, 0.03739, 0.046...","[0.0, 0.008303340279093262, 0.0194351587574825..."
3319,233,-10.40240,-0.904422,0.000000,"[1.0, 0.95022, 0.90043, 0.85057, 0.80065, 0.75...","[0.0, 0.00789, 0.0175, 0.02755, 0.03739, 0.046...","[0.0, 0.008303340279093262, 0.0194351587574825..."
3320,233,-11.07290,-0.985190,0.000000,"[1.0, 0.95022, 0.90043, 0.85057, 0.80065, 0.75...","[0.0, 0.00789, 0.0175, 0.02755, 0.03739, 0.046...","[0.0, 0.008303340279093262, 0.0194351587574825..."
3321,233,-12.35700,-0.646564,0.009324,"[1.0, 0.95022, 0.90043, 0.85057, 0.80065, 0.75...","[0.0, 0.00789, 0.0175, 0.02755, 0.03739, 0.046...","[0.0, 0.008303340279093262, 0.0194351587574825..."


## Plotting some random airfoils

In [555]:
df2['ysr'].apply(len).unique()

array([51, 35, 97, 34, 50, 36, 61])

Here is the problem. Each airfoil feometry consists of different number of points. What I decided to do is to shrink the number of points to the lowest number. And remove nan and inf values as well.

In [556]:
df2['ysr'] = df2['ysr'].apply(lambda x: x[~np.isnan(x)]) 
df2['ysr'] = df2['ysr'].apply(lambda x: x[~np.isinf(x)]) 
df2['ysr'].apply(len).unique()

array([50, 34, 96, 33, 49, 35, 60])

In [557]:
def shrink(lst, num = 33):
    while len(lst)!=num:
        lst=np.delete(lst, random.randint(0, len(lst)-1))
    return lst

In [558]:
df2['ysr'] = df2['ysr'].apply(lambda x: shrink(x))

In [559]:
df2[[f'ysr{i}' for i in range(33)]] = pd.DataFrame(df2.ysr.tolist(), index= df2.index)
df2

,id,alpha,Cl,Cd,xs,ys,ysr,ysr0,ysr1,ysr2,...,ysr23,ysr24,ysr25,ysr26,ysr27,ysr28,ysr29,ysr30,ysr31,ysr32
0,216,19.37620,1.373150,0.017005,"[1.0, 0.9502, 0.9004, 0.85055, 0.80063, 0.7506...","[0.0, 0.00744, 0.0166, 0.02649, 0.03653, 0.046...","[0.01843625055530875, 0.031144553524190228, 0....",0.018436,0.031145,0.061786,...,-0.160482,-0.140636,-0.121300,-0.102312,-0.084171,-0.038006,-0.015339,-0.006959,-0.001179,0.000000
1,216,18.29290,1.411050,0.018944,"[1.0, 0.9502, 0.9004, 0.85055, 0.80063, 0.7506...","[0.0, 0.00744, 0.0166, 0.02649, 0.03653, 0.046...","[0.0, 0.01843625055530875, 0.04562656907685198...",0.000000,0.018436,0.045627,...,-0.269454,-0.233617,-0.181507,-0.160482,-0.067332,-0.051875,-0.038006,-0.025783,-0.015339,-0.006959
2,216,17.40130,1.519540,0.000000,"[1.0, 0.9502, 0.9004, 0.85055, 0.80063, 0.7506...","[0.0, 0.00744, 0.0166, 0.02649, 0.03653, 0.046...","[0.0, 0.00782993054093875, 0.04562656907685198...",0.000000,0.007830,0.045627,...,-0.233617,-0.205310,-0.181507,-0.140636,-0.102312,-0.084171,-0.067332,-0.025783,-0.015339,0.000000
3,216,16.28550,1.549260,0.000000,"[1.0, 0.9502, 0.9004, 0.85055, 0.80063, 0.7506...","[0.0, 0.00744, 0.0166, 0.02649, 0.03653, 0.046...","[0.0, 0.01843625055530875, 0.04562656907685198...",0.000000,0.018436,0.045627,...,-0.160482,-0.140636,-0.121300,-0.067332,-0.038006,-0.025783,-0.015339,-0.006959,-0.001179,0.000000
4,216,15.29650,1.531330,0.000000,"[1.0, 0.9502, 0.9004, 0.85055, 0.80063, 0.7506...","[0.0, 0.00744, 0.0166, 0.02649, 0.03653, 0.046...","[0.0, 0.00782993054093875, 0.01843625055530875...",0.000000,0.007830,0.018436,...,-0.140636,-0.121300,-0.102312,-0.084171,-0.051875,-0.038006,-0.025783,-0.015339,-0.001179,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3318,233,-8.23054,-0.714753,0.010010,"[1.0, 0.95022, 0.90043, 0.85057, 0.80065, 0.75...","[0.0, 0.00789, 0.0175, 0.02755, 0.03739, 0.046...","[0.0, 0.008303340279093262, 0.0323900443232185...",0.000000,0.008303,0.032390,...,-0.173043,-0.097240,-0.086230,-0.063265,-0.050658,-0.038314,-0.016588,-0.007959,-0.001653,0.000000
3319,233,-10.40240,-0.904422,0.000000,"[1.0, 0.95022, 0.90043, 0.85057, 0.80065, 0.75...","[0.0, 0.00789, 0.0175, 0.02755, 0.03739, 0.046...","[0.01943515875748254, 0.03239004432321855, 0.0...",0.019435,0.032390,0.046700,...,-0.121784,-0.108954,-0.086230,-0.075329,-0.050658,-0.038314,-0.016588,-0.007959,-0.001653,0.000000
3320,233,-11.07290,-0.985190,0.000000,"[1.0, 0.95022, 0.90043, 0.85057, 0.80065, 0.75...","[0.0, 0.00789, 0.0175, 0.02755, 0.03739, 0.046...","[0.04669955661025417, 0.06209202568406469, 0.0...",0.046700,0.062092,0.078317,...,-0.152785,-0.136150,-0.108954,-0.097240,-0.086230,-0.063265,-0.050658,-0.026859,-0.016588,-0.007959
3321,233,-12.35700,-0.646564,0.009324,"[1.0, 0.95022, 0.90043, 0.85057, 0.80065, 0.75...","[0.0, 0.00789, 0.0175, 0.02755, 0.03739, 0.046...","[0.03239004432321855, 0.04669955661025417, 0.0...",0.032390,0.046700,0.078317,...,-0.199145,-0.152785,-0.136150,-0.121784,-0.108954,-0.097240,-0.086230,-0.075329,-0.050658,0.000000


In [561]:
Xy = df2[['alpha']+[f'ysr{i}' for i in range(33)]+['Cl']]
Xy.head()

,alpha,ysr0,ysr1,ysr2,ysr3,ysr4,ysr5,ysr6,ysr7,ysr8,...,ysr24,ysr25,ysr26,ysr27,ysr28,ysr29,ysr30,ysr31,ysr32,Cl
0,19.3762,0.018436,0.031145,0.061786,0.079530,0.098889,0.119679,0.142037,0.165420,0.241246,...,-0.140636,-0.121300,-0.102312,-0.084171,-0.038006,-0.015339,-0.006959,-0.001179,0.000000,1.37315
1,18.2929,0.000000,0.018436,0.045627,0.061786,0.098889,0.119679,0.142037,0.165420,0.189479,...,-0.233617,-0.181507,-0.160482,-0.067332,-0.051875,-0.038006,-0.025783,-0.015339,-0.006959,1.41105
2,17.4013,0.000000,0.007830,0.045627,0.061786,0.098889,0.119679,0.142037,0.189479,0.241246,...,-0.205310,-0.181507,-0.140636,-0.102312,-0.084171,-0.067332,-0.025783,-0.015339,0.000000,1.51954
3,16.2855,0.000000,0.018436,0.045627,0.061786,0.079530,0.098889,0.119679,0.142037,0.165420,...,-0.140636,-0.121300,-0.067332,-0.038006,-0.025783,-0.015339,-0.006959,-0.001179,0.000000,1.54926
4,15.2965,0.000000,0.007830,0.018436,0.031145,0.045627,0.079530,0.098889,0.142037,0.165420,...,-0.121300,-0.102312,-0.084171,-0.051875,-0.038006,-0.025783,-0.015339,-0.001179,0.000000,1.53133


In [597]:
mms= MinMaxScaler()
X = Xy.iloc[:,:-1]
X = mms.fit_transform(X)
df2 = pd.DataFrame(df2)
rfr = RandomForestRegressor(n_estimators=300)
X_train, X_test, y_train, y_test = train_test_split(X, Xy.iloc[:,-1])

In [598]:
rfr.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=300, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [599]:
print(np.mean(y_train),'±',mean_squared_error(y_train,rfr.predict(X_train), squared=False))
print(np.mean(y_test),'±',mean_squared_error(y_test,rfr.predict(X_test), squared=False))

0.27887979777777805 ± 0.050086799793501766
0.2415531666666667 ± 0.1430606140668313


In [587]:
from sklearn.model_selection import cross_val_score
cross_val_score(rfr,X_test, y_test)

array([0.9723972 , 0.98184397, 0.97103455, 0.97074909, 0.97326944])

In [596]:
from collections import defaultdict
dct = {'feature':Xy.columns[:-1], 'importance':rfr.feature_importances_}
dfFI = pd.DataFrame(dct)
dfFI.sort_values(by='importance', ascending = False)

,feature,importance
0,alpha,0.969911
32,ysr31,0.003258
8,ysr7,0.002867
21,ysr20,0.001529
22,ysr21,0.001499
23,ysr22,0.001191
2,ysr1,0.001123
10,ysr9,0.001105
17,ysr16,0.000990
18,ysr17,0.000872


In [600]:
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
rf = RandomForestRegressor()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, 
                               param_distributions = random_grid, 
                               n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(X_train, y_train)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    7.9s
/opt/anaconda3/lib/python3.7/site-packages/joblib/externals/loky/process_executor.py:706: UserWarning:

A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.

[Parallel(n_jobs=-1)]: Done 130 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  5.2min finished


RandomizedSearchCV(cv=3, error_score=nan,
                   estimator=RandomForestRegressor(bootstrap=True,
                                                   ccp_alpha=0.0,
                                                   criterion='mse',
                                                   max_depth=None,
                                                   max_features='auto',
                                                   max_leaf_nodes=None,
                                                   max_samples=None,
                                                   min_impurity_decrease=0.0,
                                                   min_impurity_split=None,
                                                   min_samples_leaf=1,
                                                   min_samples_split=2,
                                                   min_weight_fraction_leaf=0.0,
                                                   n_estimators=100,
                              

In [602]:
rfr = rf_random.best_estimator_
print(np.mean(y_train),'±',mean_squared_error(y_train,rfr.predict(X_train), squared=False))
print(np.mean(y_test),'±',mean_squared_error(y_test,rfr.predict(X_test), squared=False))

0.27887979777777805 ± 0.056433614435631774
0.2415531666666667 ± 0.1427659285692798


In [603]:
rf_random.best_params_

{'n_estimators': 600,
 'min_samples_split': 5,
 'min_samples_leaf': 1,
 'max_features': 'auto',
 'max_depth': 80,
 'bootstrap': True}